In [1]:
from rmgpy import settings
import os
import numpy as np
from rmgpy.data.rmg import RMGDatabase
from rmgpy.exceptions import ActionError
from rmgpy.chemkin import save_species_dictionary
from collections import defaultdict,OrderedDict
from rmgpy.kinetics import Arrhenius, ArrheniusEP, SurfaceArrhenius, SurfaceArrheniusBEP, StickingCoefficient, \
                           MultiArrhenius,StickingCoefficientBEP
from rmgpy.data.kinetics import KineticsDepository
import logging
import pandas as pd
from copy import copy

In [2]:
database = RMGDatabase()

In [3]:
database.load(
    settings['database.directory'],
    kinetics_families='all',
    thermo_libraries=[],
    reaction_libraries=['2-BTP'],
    solvation=False,
    testing=False,
)

In [4]:
def add_mols_to_species(species,mols):
    
    for spc in species:
        for mol in mols:
            if spc.is_isomorphic(mol):
                spc.molecule = [mol.copy(deep=True)]
                break

In [5]:
def match_reaction_to_family(entry,families={}):
    
    reaction = entry.item
    
    n_reacts = len(reaction.reactants)
    n_prods = len(reaction.products)
    rs = [r.molecule[0] for r in reaction.reactants]
    ps = [p.molecule[0] for p in reaction.products]
    
    for mol in rs + ps:
        mol.clear_labeled_atoms()
    
    # can't match rxn with more than 3 reacts
    if n_reacts > 3:
        return None
    
    labeled_rs = labeled_ps = None
    # try to match in the forward direction first
    for family_name, reaction_family in families.items():
        
        # make sure template stochiometry matches rxn stoichometry
        if (len(reaction_family.forward_template.reactants) == n_reacts) and \
            (len(reaction_family.forward_template.products) == n_prods):
            
            try:
                labeled_rs, labeled_ps = \
                reaction_family.get_labeled_reactants_and_products(rs,ps,relabel_atoms=False)
            except (ActionError, ValueError):
                continue
            
            if labeled_rs is not None: # found a match!            
                add_mols_to_species(reaction.reactants,labeled_rs)
                add_mols_to_species(reaction.products,labeled_ps)
                reaction.comment = f'matches {family_name} family in forward direction'
                entry.long_desc += f'matches {family_name} family in forward direction\n'
                entry.direction = 'forward'
                return (family_name,'forward')
            
            
    if n_prods > 3:
        return None
    
    for family_name, reaction_family in families.items():
        
        
        if (len(reaction_family.forward_template.reactants) == n_prods) and \
            (len(reaction_family.forward_template.products) == n_reacts):
            
            try:
                labeled_ps, labeled_rs = \
                reaction_family.get_labeled_reactants_and_products(ps,rs,relabel_atoms=False)
            except (ActionError, ValueError):
                continue
            
            if labeled_rs:
                
                add_mols_to_species(reaction.reactants,labeled_rs)
                add_mols_to_species(reaction.products,labeled_ps)
                reaction.comment = f'matches {family_name} family in reverse direction'
                entry.long_desc += f'matches {family_name} family in reverse direction'
                entry.direction = 'reverse'
                return (family_name, 'reverse')
    
    # No match
    entry.long_desc += 'Does not match an rmg reaction family\n'
    return None

In [6]:
def match_library_reactions_to_families(library,families={}):
    
    assert len(library.entries) > 0
    
    family_entries_dict = defaultdict(list)
    
    for index, entry in library.entries.items():
        match = match_reaction_to_family(entry,families=families)
        if match is not None:
            family, direction = match
            print(f"***** reaction {index} {entry.item} matched {family} in {direction} direction *****")
            family_entries_dict[family].append(entry)
        else:
            print(f"No match for {index} {entry.item}")
            
    return family_entries_dict
    

In [7]:
def get_library(name=''):
    
    if name not in database.kinetics.libraries:
        raise KeyError(f"{name} not in kinetics libraries database")
    library = database.kinetics.libraries[name]
    for entry in library.entries.values():
        entry.item.reactants = [spcs.copy(deep=True) for spcs in entry.item.reactants]
        entry.item.products = [spcs.copy(deep=True) for spcs in entry.item.products]
    
    return library

In [8]:
def check_kinetics(kinetics):
    

    supported_classes = (Arrhenius, ArrheniusEP, SurfaceArrhenius, SurfaceArrheniusBEP, StickingCoefficient,
                        StickingCoefficientBEP)
    
    if type(kinetics) in supported_classes:
        return True
    
            

In [9]:
def remove_unsupported_kinetic_classes(entries: list):

    unsupported = []
    supported = []
    for entry in entries:
        if check_kinetics(entry.data):
            supported.append(entry)
        else:
            entry.long_desc += f'Cannot add kinetics class {type(entry.data)} to training\n'
            unsupported.append(entry)
   
    return supported,unsupported

In [10]:
def get_training_species_dict(training_depository):
    
    path = os.path.join(settings['database.directory'],'kinetics',
            'families',training_depository.name,'dictionary.txt') 
    
    if not os.path.exists(path):
        logging.warning(f"{path} does not exist")
        logging.warning(f"making a new species dictionary")
        return OrderedDict()
    else:
        return training_depository.get_species(path, resonance=False)
    

In [11]:
def get_training_depository(family=''):
    
    depos = [depo for depo in database.kinetics.families[family].depositories if 'training' in depo.name.lower()]
    
    if len(depos) == 1:
        return depos[0]
    elif len(depos) == 0:
        logging.warning(f"Could not find training depository for {family}, making a new one")
        depo = KineticsDepository(name=f'{family}/training')
        return depo
    else:
        logging.error("more than 1 training depo?")
        raise
        

In [12]:
def add_reaction_species_to_species_dict(reaction,species_dict):
    
    
    for spcs in reaction.reactants + reaction.products:
        add_to_training = True
        for label,training_spc in species_dict.items():
            if spcs.is_isomorphic(training_spc,generate_initial_map=True):
                print(f"{spcs} matches {training_spc}")
                spcs.label = label
                add_to_training = False
                break
        if add_to_training:
            labels = sorted([label.strip('*') for label in spcs.molecule[0].get_all_labeled_atoms().keys()])
            spcs.label += '_'
            for l in labels:
                spcs.label += l
            if spcs.label in species_dict:
                while spcs.label in species_dict:
                    spcs.label += '_'
            species_dict[spcs.label] = spcs.copy(deep=True)
                


In [13]:
def identify_duplicates(entries,training_depository,compare=True):
    
    unique_entries = []
    duplicates_entries = []
    comparisons = []
    
    for entry in entries:
        unique = True
        reaction = entry.item
        for i,training_entry in training_depository.entries.items():
            if reaction.is_isomorphic(training_entry.item,generate_initial_map=False):
                print(f"{entry.index} {entry} is isomorphic to training rxn {i} {training_entry}")
                unique = False
                kinetics = entry.data
                training_kinetics = training_entry.data
                duplicates_entries.append(entry)
                entry.long_desc += f'Isomorphic to {training_entry.item.to_labeled_str()} in {training_depository.name}\n'
                if compare:
                    try:
                        comparison = {}
                        comparison['family'] = training_depository.name
                        comparison['reaction'] = entry.item.to_labeled_str()
                        comparison['index'] = entry.index
                        comparison['kinetics'] = kinetics.__repr__()
                        comparison['training_kinetics'] = training_kinetics.__repr__()
                        comparison['training_reaction'] = training_entry.item.to_labeled_str()
                        comparison['training_reaction_index'] = training_entry.index
                        comparison['is_similar'] = kinetics.is_similar_to(training_kinetics)
                        for T in np.arange(500,2001,500):
                            k_r = kinetics.get_rate_coefficient(T)
                            k_train = training_kinetics.get_rate_coefficient(T)
                            ratio = k_r/k_train
                            comparison[f'reaction_rate_coeff_{T:d}K'] = k_r
                            comparison[f'training_reaction_rate_coeff_{T:d}K'] = k_train
                            comparison[f'reaction_k/training_k_{T:d}:'] = ratio
                            entry.long_desc += f'k_r/k_train at {T:d}K = {ratio}\n'
                        comparisons.append(comparison)
                    except:
                        print(f'failed to compare kinetics for {entry} and {training_entry}')
                break
        if unique:
            unique_entries.append(entry)
            
    return unique_entries, duplicates_entries, pd.DataFrame(comparisons)
            

In [14]:
def save_df_to_csv(df,path):
    
    df = df[sorted(df.columns)]
    if not os.path.exists(os.path.dirname(path)):
        os.makedirs(path,exist_ok=True)
    if not os.path.exists(path):
        df.to_csv(path,index=False)
    else:
        df.to_csv(path,index=False,mode='a',header=False)

In [15]:
def add_to_training(entries, family='', reaction_library='', add_duplicates=False, compare=True):
    
    added_to_training_indicies = []
    training_depository = get_training_depository(family)
    species_dict = get_training_species_dict(training_depository)
    
    supported,unsupported = remove_unsupported_kinetic_classes(entries)
    
    unique_entries, duplicates_entries, comparisons_df = \
        identify_duplicates(supported,training_depository,compare=compare)
    
    for entry in supported:
        if add_duplicates:
            entries_to_add = unique_entries + duplicates_entries
        else:
            entries_to_add = unique_entries
        
    max_i = max(training_depository.entries.keys())
    for i,entry in enumerate(entries_to_add):
        library_index = copy(entry.index)
        added_to_training_indicies.append(library_index)
        i += 1        
        add_reaction_species_to_species_dict(entry.item,species_dict)
        entry.label = entry.item.to_labeled_str()
        entry.short_desc = f'Reaction {library_index} from reaction library {reaction_library}'
        entry.index = max_i + i
        training_depository.entries[max_i + i] = entry
        
        
    return species_dict, training_depository, comparisons_df, added_to_training_indicies
        
    

In [16]:
library = get_library('2-BTP')
matched_family_entries_dict = match_library_reactions_to_families(library,families=database.kinetics.families)

No match for 0 H + O2 <=> O + OH
***** reaction 1 O + H2 <=> H + OH matched H_Abstraction in forward direction *****
***** reaction 2 OH + H2 <=> H + H2O matched H_Abstraction in forward direction *****
***** reaction 3 OH + OH <=> O + H2O matched H_Abstraction in forward direction *****
No match for 4 H + H <=> H2
No match for 5 H + H + H2 <=> H2 + H2
No match for 6 H + H + H2O <=> H2 + H2O
No match for 7 H + H + CO2 <=> H2 + CO2
No match for 8 H + OH <=> H2O
***** reaction 9 O + H <=> OH matched Birad_R_Recombination in forward direction *****
No match for 10 O + O <=> O2
No match for 11 H + O2 <=> HO2
***** reaction 12 H2 + O2 <=> HO2 + H matched H_Abstraction in forward direction *****
No match for 13 OH + OH <=> H2O2
***** reaction 14 HO2 + H <=> O + H2O matched Substitution_O in forward direction *****
No match for 15 HO2 + H <=> OH + OH
***** reaction 16 HO2 + O <=> OH + O2 matched H_Abstraction in forward direction *****
***** reaction 17 HO2 + HO2 <=> O2 + H2O2 matched H_Abstr

No match for 192 C2H3 + C2H3 <=> iC4H5 + H
No match for 193 C2H3 + C2H3 <=> nC4H5 + H
***** reaction 194 C2H3 + C2H3 <=> C2H2 + C2H4 matched Disproportionation in forward direction *****
No match for 195 CH2CHO <=> CH3 + CO
No match for 196 CH2CHO + H <=> CH3CHO
No match for 197 CH2CHO + H <=> CH3CO + H
No match for 198 CH2CHO + H <=> CH3 + HCO
***** reaction 199 CH2CHO + H <=> CH2CO + H2 matched Disproportionation in forward direction *****
***** reaction 200 CH2CHO + O <=> CH2CO + OH matched Disproportionation in forward direction *****
***** reaction 201 CH2CHO + OH <=> CH2CO + H2O matched Disproportionation in forward direction *****
***** reaction 202 CH2CHO + O2 <=> CH2CO + HO2 matched Disproportionation in forward direction *****
No match for 203 CH2CHO + O2 <=> CH2O + CO + OH
***** reaction 204 CH3 + CO <=> CH3CO matched R_Addition_COm in forward direction *****
No match for 205 CH3CO + H <=> CH3CHO
No match for 206 CH3CO + H <=> CH3 + HCO
***** reaction 207 CH3CO + O <=> CH2CO

***** reaction 325 CH3CCH2 <=> CH3CHCH matched intra_H_migration in forward direction *****
***** reaction 326 CH3CCH2 + H <=> pC3H4 + H2 matched Disproportionation in forward direction *****
No match for 327 CH3CCH2 + O <=> CH3 + CH2CO
No match for 328 CH3CCH2 + OH <=> CH3 + CH2CO + H
No match for 329 CH3CCH2 + O2 <=> CH3CO + CH2O
No match for 330 CH3CCH2 + HO2 <=> CH3 + CH2CO + OH
***** reaction 331 CH3CCH2 + HCO <=> C3H6 + CO matched CO_Disproportionation in forward direction *****
***** reaction 332 CH3CCH2 + CH3 <=> pC3H4 + CH4 matched Disproportionation in forward direction *****
No match for 333 CH3CCH2 + CH3 <=> iC4H8
***** reaction 334 CH3CHCH + H <=> pC3H4 + H2 matched Disproportionation in forward direction *****
No match for 335 CH3CHCH + O <=> C2H4 + HCO
No match for 336 CH3CHCH + OH <=> C2H4 + HCO + H
No match for 337 CH3CHCH + O2 <=> CH3CHO + HCO
No match for 338 CH3CHCH + HO2 <=> C2H4 + HCO + OH
***** reaction 339 CH3CHCH + HCO <=> C3H6 + CO matched CO_Disproportionatio

No match for 459 C4H5-2 + C2H4 <=> C5H6 + CH3
No match for 460 C4H6 <=> iC4H5 + H
No match for 461 C4H6 <=> nC4H5 + H
No match for 462 C4H6 <=> C4H4 + H2
***** reaction 463 C4H6 + H <=> nC4H5 + H2 matched H_Abstraction in forward direction *****
***** reaction 464 C4H6 + H <=> iC4H5 + H2 matched H_Abstraction in forward direction *****
No match for 465 C4H6 + H <=> C2H4 + C2H3
No match for 466 C4H6 + H <=> pC3H4 + CH3
No match for 467 C4H6 + H <=> aC3H4 + CH3
***** reaction 468 C4H6 + O <=> nC4H5 + OH matched H_Abstraction in forward direction *****
***** reaction 469 C4H6 + O <=> iC4H5 + OH matched H_Abstraction in forward direction *****
No match for 470 C4H6 + O <=> CH3CHCHCO + H
No match for 471 C4H6 + O <=> CH2CHCHCHO + H
***** reaction 472 C4H6 + OH <=> nC4H5 + H2O matched H_Abstraction in forward direction *****
***** reaction 473 C4H6 + OH <=> iC4H5 + H2O matched H_Abstraction in forward direction *****
No match for 474 C4H6 + HO2 <=> C4H6O25 + OH
No match for 475 C4H6 + HO2 <=

***** reaction 629 iC4H10 + O <=> tC4H9 + OH matched H_Abstraction in forward direction *****
***** reaction 630 iC4H10 + OH <=> iC4H9 + H2O matched H_Abstraction in forward direction *****
***** reaction 631 iC4H10 + OH <=> tC4H9 + H2O matched H_Abstraction in forward direction *****
***** reaction 632 iC4H10 + HO2 <=> iC4H9 + H2O2 matched H_Abstraction in forward direction *****
***** reaction 633 iC4H10 + HO2 <=> tC4H9 + H2O2 matched H_Abstraction in forward direction *****
***** reaction 634 iC4H10 + O2 <=> iC4H9 + HO2 matched H_Abstraction in forward direction *****
***** reaction 635 iC4H10 + O2 <=> tC4H9 + HO2 matched H_Abstraction in forward direction *****
***** reaction 636 iC4H10 + CH3 <=> iC4H9 + CH4 matched H_Abstraction in forward direction *****
***** reaction 637 iC4H10 + CH3 <=> tC4H9 + CH4 matched H_Abstraction in forward direction *****
***** reaction 638 C6H2 + H <=> C6H3 matched R_Addition_MultipleBond in forward direction *****
No match for 639 C6H3 + H <=> C4H2 +

...

No match for 979 CH3-CHF + H <=> CH3 + CH2F
No match for 980 CH2F-CHF + H <=> CH2F + CH2F
No match for 981 CHF2-CH2 + H <=> CH3 + CHF2
No match for 982 CH3-CF2 + H <=> CH3 + CHF2
No match for 983 CHF2-CHF + H <=> CH2F + CHF2
No match for 984 CH2F-CF2 + H <=> CH2F + CHF2
No match for 985 CF3-CH2 + H <=> CH3 + CF3
No match for 986 CF3-CHF + H <=> CF3 + CH2F
No match for 987 CHF2-CF2 + H <=> CHF2 + CHF2
No match for 988 CF3-CF2 + H <=> CHF2 + CF3
No match for 989 CH2F-CH2 + H <=> CH3-CH2F
No match for 990 CH3-CHF + H <=> CH3-CH2F
No match for 991 CH2F-CHF + H <=> CH2F-CH2F
No match for 992 CHF2-CH2 + H <=> CH3-CHF2
No match for 993 CH3-CF2 + H <=> CH3-CHF2
No match for 994 CHF2-CHF + H <=> CH2F-CHF2
No match for 995 CH2F-CF2 + H <=> CH2F-CHF2
No match for 996 CF3-CH2 + H <=> CH3-CF3
No match for 997 CHF2-CF2 + H <=> CHF2-CHF2
No match for 998 CF3-CF2 + H <=> CHF2-CF3
No match for 999 CH3 + CH2F <=> C2H4 + HF
No match for 1000 CH2F + CH2F <=> CH2:CHF + HF
No match for 1001 CH3 + CHF2 <=> C

***** reaction 1122 CHF2-CHF2 + CH2F <=> CHF2-CF2 + CH3F matched H_Abstraction in forward direction *****
***** reaction 1123 CHF2-CF3 + CH2F <=> CF3-CF2 + CH3F matched H_Abstraction in forward direction *****
***** reaction 1124 CH3-CH2F + CHF2 <=> CH2F-CH2 + CH2F2 matched H_Abstraction in forward direction *****
***** reaction 1125 CH3-CH2F + CHF2 <=> CH3-CHF + CH2F2 matched H_Abstraction in forward direction *****
***** reaction 1126 CH3-CHF2 + CHF2 <=> CHF2-CH2 + CH2F2 matched H_Abstraction in forward direction *****
***** reaction 1127 CH3-CHF2 + CHF2 <=> CH3-CF2 + CH2F2 matched H_Abstraction in forward direction *****
***** reaction 1128 CH3-CF3 + CHF2 <=> CF3-CH2 + CH2F2 matched H_Abstraction in forward direction *****
***** reaction 1129 CH2F-CH2F + CHF2 <=> CH2F-CHF + CH2F2 matched H_Abstraction in forward direction *****
***** reaction 1130 CH2F-CHF2 + CHF2 <=> CHF2-CHF + CH2F2 matched H_Abstraction in forward direction *****
***** reaction 1131 CH2F-CHF2 + CHF2 <=> CH2F-CF2 

***** reaction 1227 CH2F-CH2 + HO2 <=> CH3-CH2F + O2 matched H_Abstraction in forward direction *****
***** reaction 1228 CH2F-CHF + HO2 <=> CH2F-CH2F + O2 matched H_Abstraction in forward direction *****
***** reaction 1229 CH2F-CF2 + HO2 <=> CH2F-CHF2 + O2 matched H_Abstraction in forward direction *****
***** reaction 1230 CHF2-CH2 + HO2 <=> CH3-CHF2 + O2 matched H_Abstraction in forward direction *****
***** reaction 1231 CHF2-CHF + HO2 <=> CH2F-CHF2 + O2 matched H_Abstraction in forward direction *****
***** reaction 1232 CHF2-CF2 + HO2 <=> CHF2-CHF2 + O2 matched H_Abstraction in forward direction *****
***** reaction 1233 CF3-CH2 + HO2 <=> CH3-CF3 + O2 matched H_Abstraction in forward direction *****
***** reaction 1234 CF3-CHF + HO2 <=> CH2F-CF3 + O2 matched H_Abstraction in forward direction *****
***** reaction 1235 CF3-CF2 + HO2 <=> CHF2-CF3 + O2 matched H_Abstraction in forward direction *****
***** reaction 1236 CH3-CHF + CH2O <=> CH3-CH2F + HCO matched H_Abstraction in for

***** reaction 1389 CF3CHO + OH <=> CF3CO + H2O matched H_Abstraction in forward direction *****
***** reaction 1390 CF3CHO + O <=> CF3CO + OH matched H_Abstraction in forward direction *****
***** reaction 1391 CF3CHO + CH3 <=> CF3CO + CH4 matched H_Abstraction in forward direction *****
***** reaction 1392 CF3CHO + CF3 <=> CHF3 + CF3CO matched H_Abstraction in forward direction *****
No match for 1393 CF3CHO <=> CF2CO + HF
***** reaction 1394 CF3CHO + F <=> CF3CO + HF matched H_Abstraction in forward direction *****
No match for 1395 CF3CHO <=> CF3 + HCO
***** reaction 1396 CF3COF + H <=> CF3CO + HF matched F_Abstraction in forward direction *****
No match for 1397 CF3COF <=> CF3 + CF:O
***** reaction 1398 CF3COF + CF3 <=> CF3CO + CF4 matched F_Abstraction in forward direction *****
***** reaction 1399 CF3COF + CF3-CF2 <=> CF3CO + CF3-CF3 matched F_Abstraction in forward direction *****
***** reaction 1400 CF3-CF3 + CF3 <=> CF4 + CF3-CF2 matched F_Abstraction in forward direction ***

No match for 1519 CH3 + BRO <=> CH2O + HBR
***** reaction 1520 CH3O + BRO <=> CH2O + BROH matched Disproportionation in forward direction *****
***** reaction 1521 CH3O + HBR <=> CH3OH + BR matched H_Abstraction in forward direction *****
No match for 1522 BR + CH2 <=> CH + HBR
***** reaction 1523 F + CH3BR <=> HF + CH2BR matched H_Abstraction in forward direction *****
No match for 1524 CF3CCH + HBR <=> BTP
No match for 1525 BTP <=> CF3CCH2 + BR
No match for 1526 BTP + H <=> CF3CHCH2 + BR
***** reaction 1527 BTP + H <=> CF3CHBRCH2 matched R_Addition_MultipleBond in forward direction *****
No match for 1528 CF3CHCH2 + BR <=> CF3CHBRCH2
***** reaction 1529 BTP + H <=> CF3CCH2 + HBR matched Br_Abstraction in forward direction *****
No match for 1530 BTP + H <=> C2H3BR + CF3
***** reaction 1531 BTP + H <=> CF3CBRCH + H2 matched H_Abstraction in forward direction *****
No match for 1532 BTP + OH <=> CF3COCH3 + BR
No match for 1533 CF3CHCH2 + OH <=> CF3COCH3 + H
***** reaction 1534 BTP + OH

In [17]:
species_dict, training_depository, comparisons_df, added = \
add_to_training(matched_family_entries_dict['H_Abstraction'], family='H_Abstraction', reaction_library='2-BTP',
                add_duplicates=False, compare=True)

1 O + H2 <=> H + OH is isomorphic to training rxn 285 H2 + O_rad <=> HO + H
2 OH + H2 <=> H + H2O is isomorphic to training rxn 286 H2 + OH <=> H2O_p + H_p
3 OH + OH <=> O + H2O is isomorphic to training rxn 379 H2O + O_rad <=> HO + OH_p23
12 H2 + O2 <=> HO2 + H is isomorphic to training rxn 289 H2 + O2 <=> HO2_r12 + H_p
16 HO2 + O <=> OH + O2 is isomorphic to training rxn 4277 O_rad + HO2_r12 <=> O2 + HO
19 H2O2 + H <=> HO2 + H2 is isomorphic to training rxn 4276 H + H2O2 <=> H2 + HO2_r3
21 H2O2 + O <=> OH + HO2 is isomorphic to training rxn 4278 O_rad + H2O2 <=> HO2_r3 + HO
71 CH2O + H <=> HCO + H2 is isomorphic to training rxn 309 H2 + HCO_r3 <=> CH2O + H
72 CH2O + O <=> HCO + OH is isomorphic to training rxn 359 CH2O + O_rad <=> HO + HCO_r3
73 CH2O + OH <=> HCO + H2O is isomorphic to training rxn 367 OH_p23 + CH2O <=> H2O + HCO_r3
74 CH2O + O2 <=> HCO + HO2 is isomorphic to training rxn 358 CH2O + O2 <=> HO2_r12 + HCO_r3
75 CH2O + HO2 <=> HCO + H2O2 is isomorphic to training rxn 36

767 C2H5OH + OH <=> C2H4OH + H2O is isomorphic to training rxn 227 C2H6O-2 + OH <=> CH2CH2OH_p + H2O_p
768 C2H5OH + OH <=> CH3CHOH + H2O is isomorphic to training rxn 226 C2H6O + OH <=> CH3CHOH_p + H2O_p
770 C2H5OH + H <=> C2H4OH + H2 is isomorphic to training rxn 221 C2H6O-2 + H <=> CH2CH2OH_p + H2_p
771 C2H5OH + H <=> CH3CHOH + H2 is isomorphic to training rxn 220 C2H6O + H <=> CH3CHOH_p + H2_p
772 C2H5OH + H <=> CH3CH2O + H2 is isomorphic to training rxn 222 CH3CH2OH_rO + H <=> CH3CH2O_p + H2_p
773 C2H5OH + O <=> C2H4OH + OH is isomorphic to training rxn 223 C2H6O-2 + O_rad <=> CH2CH2OH_p + OH_p23
774 C2H5OH + O <=> CH3CHOH + OH is isomorphic to training rxn 224 C2H6O + O_rad <=> CH3CHOH_p + OH_p23
775 C2H5OH + O <=> CH3CH2O + OH is isomorphic to training rxn 225 CH3CH2OH_rO + O_rad <=> CH3CH2O_p + OH_p23
776 C2H5OH + CH3 <=> C2H4OH + CH4 is isomorphic to training rxn 232 C2H6O-2 + CH3_r3 <=> CH2CH2OH_p + CH4
777 C2H5OH + CH3 <=> CH3CHOH + CH4 is isomorphic to training rxn 231 C2H6O

1075 CHF2-CF3 + OH <=> CF3-CF2 + H2O is isomorphic to training rxn 4086 CHF2-CF3 + OH <=> CF3-CF2_p1 + H2O_p
1076 CH3-CHF + H2O2 <=> CH3-CH2F + HO2 is isomorphic to training rxn 4087 CH3-CHF + H2O2 <=> CH3-CH2F_p23 + HO2
1077 CH3-CF2 + H2O2 <=> CH3-CHF2 + HO2 is isomorphic to training rxn 4088 CH3-CF2 + H2O2 <=> CH3-CHF2_p23 + HO2
1078 CH2F-CH2 + H2O2 <=> CH3-CH2F + HO2 is isomorphic to training rxn 4089 CH2F-CH2 + H2O2 <=> CH3-CH2F_p23 + HO2
1079 CH2F-CHF + H2O2 <=> CH2F-CH2F + HO2 is isomorphic to training rxn 4090 CH2F-CHF + H2O2 <=> CH2F-CH2F_p23 + HO2
1080 CH2F-CF2 + H2O2 <=> CH2F-CHF2 + HO2 is isomorphic to training rxn 4091 CH2F-CF2 + H2O2 <=> CH2F-CHF2_p23 + HO2
1081 CHF2-CH2 + H2O2 <=> CH3-CHF2 + HO2 is isomorphic to training rxn 4092 CHF2-CH2 + H2O2 <=> CH3-CHF2_p23 + HO2
1082 CHF2-CHF + H2O2 <=> CH2F-CHF2 + HO2 is isomorphic to training rxn 4093 CHF2-CHF + H2O2 <=> CH2F-CHF2_p23 + HO2
1083 CHF2-CF2 + H2O2 <=> CHF2-CHF2 + HO2 is isomorphic to training rxn 4094 CHF2-CF2 + H2O2

1233 CF3-CH2 + HO2 <=> CH3-CF3 + O2 is isomorphic to training rxn 4162 CF3-CH2 + HO2_r12 <=> CH3-CF3 + O2
1234 CF3-CHF + HO2 <=> CH2F-CF3 + O2 is isomorphic to training rxn 4163 CF3-CHF + HO2_r12 <=> CH2F-CF3 + O2
1235 CF3-CF2 + HO2 <=> CHF2-CF3 + O2 is isomorphic to training rxn 4164 CF3-CF2 + HO2_r12 <=> CHF2-CF3 + O2
1236 CH3-CHF + CH2O <=> CH3-CH2F + HCO is isomorphic to training rxn 4165 CH3-CHF + CH2O <=> CH3-CH2F_p23 + CHO_p1
1237 CH3-CF2 + CH2O <=> CH3-CHF2 + HCO is isomorphic to training rxn 4166 CH3-CF2 + CH2O <=> CH3-CHF2_p23 + CHO_p1
1238 CH2F-CH2 + CH2O <=> CH3-CH2F + HCO is isomorphic to training rxn 4167 CH2F-CH2 + CH2O <=> CH3-CH2F_p23 + CHO_p1
1239 CH2F-CHF + CH2O <=> CH2F-CH2F + HCO is isomorphic to training rxn 4168 CH2F-CHF + CH2O <=> CH2F-CH2F_p23 + CHO_p1
1240 CH2F-CF2 + CH2O <=> CH2F-CHF2 + HCO is isomorphic to training rxn 4169 CH2F-CF2 + CH2O <=> CH2F-CHF2_p23 + CHO_p1
1241 CHF2-CH2 + CH2O <=> CH3-CHF2 + HCO is isomorphic to training rxn 4170 CHF2-CH2 + CH2O <=

In [ ]:
save_species_dictionary('dictionary.txt',species_dict.values())

In [ ]:
training_depository.save('reactions.py')

In [ ]:
save_df_to_csv(comparisons_df,path='./compare.csv')

In [18]:
entries_not_added = OrderedDict()
for i,entry in library.entries.items():
    if i not in added:
        rxn = entry.item
        for spcs in rxn.reactants + rxn.products:
            for mol in spcs.molecule:
                mol.clear_labeled_atoms()
        entries_not_added[i] = entry

In [19]:
library.entries = entries_not_added

In [20]:
library.save('test_lib.py')

In [21]:
library.save_dictionary('test_save_dict.txt')